In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import MisrToolkit as mtk # needs python3.6
# from mpl_toolkits.basemap import Basemap # needs python 3.8?
import sys # to check python version
import os
import glob

# Extract data from MISR-estimated roughness files by using MTK library. Should use python3.6 for this purpose 

## Plot Transect 

In [2]:
atm_file = "3cams_insample_atm.csv"
atm_dir = "/home/ehsan/misr_lab/MISR-SeaIceRoughness/post_processing"
atm_data_fp = os.path.join(atm_dir, atm_file)

In [3]:
transect_small = pd.read_csv(atm_data_fp)
transect_small

,Unnamed: 0,lat,lon,Ca_r,Cf_r,An_r,rms
0,1,76.235487,230.712293,0.66574,1.03182,0.53599,15.24
1,2,76.235648,230.710186,0.66574,1.03182,0.53599,22.32
2,3,76.235474,230.712462,0.66574,1.03182,0.53599,17.44
3,4,76.235619,230.714797,0.66574,1.03182,0.53599,14.30
4,5,76.235781,230.712690,0.66574,1.03182,0.53599,14.95
...,...,...,...,...,...,...,...
9995,9996,75.999639,230.398900,0.58862,0.95719,0.41134,5.65
9996,9997,75.999793,230.401122,0.58862,0.95719,0.41134,5.24
9997,9998,75.999951,230.399059,0.58862,0.95719,0.41134,6.27
9998,9999,76.000109,230.396997,0.58163,0.95175,0.41600,5.40


In [4]:
transect_small = transect_small.iloc[0:1000]
transect_small

,Unnamed: 0,lat,lon,Ca_r,Cf_r,An_r,rms
0,1,76.235487,230.712293,0.66574,1.03182,0.53599,15.24
1,2,76.235648,230.710186,0.66574,1.03182,0.53599,22.32
2,3,76.235474,230.712462,0.66574,1.03182,0.53599,17.44
3,4,76.235619,230.714797,0.66574,1.03182,0.53599,14.30
4,5,76.235781,230.712690,0.66574,1.03182,0.53599,14.95
...,...,...,...,...,...,...,...
995,996,76.308008,230.813090,0.56457,0.89512,0.45258,8.11
996,997,76.308171,230.810967,0.56457,0.89512,0.45258,9.46
997,998,76.308333,230.808844,0.56457,0.89512,0.45258,6.94
998,999,76.308170,230.810971,0.56457,0.89512,0.45258,9.53


In [5]:
roughness_dir = "/media/ehsan/6T_part1/2016/april_2016/14528_apr2016/project_april_2016_3cam/roughness_predicted_from_PH"


In [ ]:
# extract MISR-estimated roughness for the same latLon locations and plot them together with ATM
transect_from_misr = []
for irow in range(transect_small.shape[0]): # each row==LatLon might fall in several blocks==POB, we open each block and will average all at the end
    
    print("DF row: %s" %irow)
    lat_lon = transect_small[['lat','lon']].iloc[irow]
    print(lat_lon)
    
    path_list = mtk.latlon_to_path_list(lat_lon[0], lat_lon[1])
    print(path_list)
    
    roughness_per_pixel_list = []
    for path in path_list:
        # Q- now that we have found many paths, how figure out to select which path?
        bls = mtk.latlon_to_bls(path, 275 , lat_lon[0], lat_lon[1])
        print(bls)
        
        # use P and B to find roughness file, open it, and extract value for LS, append it to a list, and maybe finally append the list to DF as the last column
        if path < 100:
            print(path)
            path_str = str(path).zfill(3)
            print(path_str)
            
        roughness_filepattern = "roughness_toa_refl_P"+path_str+"_*"+"_B0"+str(bls[0])+".dat"
        print(roughness_filepattern)
        rough_file_fp = os.path.join(roughness_dir, roughness_filepattern)
        print(rough_file_fp)
        
        # check if file is found on machine
        roughness_file_found_list = glob.glob(os.path.join(roughness_dir, roughness_filepattern))
        print("if more than 1 file found, there is a problem with this code and it has to be reviewed")
        print(len(roughness_file_found_list))
        
        if (len(roughness_file_found_list) <= 0):
            print("roughness file not found- continue to next path")
            continue
        # open roughness file and read pixel value- chat binary format to 2D array and read pixel value
        rough_2d_arr = np.fromfile(roughness_file_found_list[0], dtype=np.double)[0:1048576].reshape((512,-1))
        print(rough_2d_arr.shape)
        
        pixel_row = bls[1]
        pixel_col = bls[2]
        roughness_value = rough_2d_arr[int(pixel_row), int(pixel_col)]
        if roughness_value > 0:
            roughness_per_pixel_list.append(roughness_value)
    
    print("========================== extracted rough pixels ===========================")
    print(roughness_per_pixel_list)
    mean_roughness_per_pixel = sum(roughness_per_pixel_list)/len(roughness_per_pixel_list)
    
    transect_from_misr.append(mean_roughness_per_pixel)


DF row: 0
lat     76.235487
lon    230.712293
Name: 0, dtype: float64
[62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148]
(29, 264.625, 9.251968383789062)
62
062
roughness_toa_refl_P062_*_B029.dat
/media/ehsan/6T_part1/2016/april_2016/14528_apr2016/project_april_2016_3cam/roughness_predicted_from_PH/roughness_toa_refl_P062_*_B029.dat
if more than 1 file found, there is a problem with this code and it has to be reviewed
1
(512, 2048)
(29, 200.7333984375, 144.29833984375)
63
063
roughness_toa_refl_P063_*_B029.dat
/media/ehsan/6T_part1/2016/april_2016/14528_apr2016/project_april_2016_3cam/roughness_predicted_from_PH/roughness_toa_refl_P063_*_B029.dat
if more than 1 file found, there is a problem with this code and it has to be reviewed
1
(512, 2048)
(29, 133.357421875, 277.6050720214844)
64
064
roughness_toa_refl_P064_*_B029.dat
/media/ehsan/6T_part1/2016/april_2016/14528_apr

if more than 1 file found, there is a problem with this code and it has to be reviewed
1
(512, 2048)
(9, 468.0986328125, 439.71014404296875)
roughness_toa_refl_P080_*_B09.dat
/media/ehsan/6T_part1/2016/april_2016/14528_apr2016/project_april_2016_3cam/roughness_predicted_from_PH/roughness_toa_refl_P080_*_B09.dat
if more than 1 file found, there is a problem with this code and it has to be reviewed
0
roughness file not found- continue to next path
(9, 396.4775390625, 308.6707763671875)
roughness_toa_refl_P080_*_B09.dat
/media/ehsan/6T_part1/2016/april_2016/14528_apr2016/project_april_2016_3cam/roughness_predicted_from_PH/roughness_toa_refl_P080_*_B09.dat
if more than 1 file found, there is a problem with this code and it has to be reviewed
0
roughness file not found- continue to next path
(9, 328.28759765625, 175.785400390625)
roughness_toa_refl_P080_*_B09.dat
/media/ehsan/6T_part1/2016/april_2016/14528_apr2016/project_april_2016_3cam/roughness_predicted_from_PH/roughness_toa_refl_P080_*

(512, 2048)
(10, 191.25439453125, 821.0343627929688)
roughness_toa_refl_P080_*_B010.dat
/media/ehsan/6T_part1/2016/april_2016/14528_apr2016/project_april_2016_3cam/roughness_predicted_from_PH/roughness_toa_refl_P080_*_B010.dat
if more than 1 file found, there is a problem with this code and it has to be reviewed
1
(512, 2048)
(10, 109.6162109375, 696.0468139648438)
roughness_toa_refl_P080_*_B010.dat
/media/ehsan/6T_part1/2016/april_2016/14528_apr2016/project_april_2016_3cam/roughness_predicted_from_PH/roughness_toa_refl_P080_*_B010.dat
if more than 1 file found, there is a problem with this code and it has to be reviewed
1
(512, 2048)
(10, 31.2666015625, 568.959228515625)
roughness_toa_refl_P080_*_B010.dat
/media/ehsan/6T_part1/2016/april_2016/14528_apr2016/project_april_2016_3cam/roughness_predicted_from_PH/roughness_toa_refl_P080_*_B010.dat
if more than 1 file found, there is a problem with this code and it has to be reviewed
1
(512, 2048)
(9, 468.255859375, 439.85540771484375)
rough

(512, 2048)
(10, 455.18310546875, 1182.44189453125)
roughness_toa_refl_P080_*_B010.dat
/media/ehsan/6T_part1/2016/april_2016/14528_apr2016/project_april_2016_3cam/roughness_predicted_from_PH/roughness_toa_refl_P080_*_B010.dat
if more than 1 file found, there is a problem with this code and it has to be reviewed
1
(512, 2048)
(10, 364.0, 1064.244384765625)
roughness_toa_refl_P080_*_B010.dat
/media/ehsan/6T_part1/2016/april_2016/14528_apr2016/project_april_2016_3cam/roughness_predicted_from_PH/roughness_toa_refl_P080_*_B010.dat
if more than 1 file found, there is a problem with this code and it has to be reviewed
1
(512, 2048)
(10, 275.94482421875, 943.700439453125)
roughness_toa_refl_P080_*_B010.dat
/media/ehsan/6T_part1/2016/april_2016/14528_apr2016/project_april_2016_3cam/roughness_predicted_from_PH/roughness_toa_refl_P080_*_B010.dat
if more than 1 file found, there is a problem with this code and it has to be reviewed
1
(512, 2048)
(10, 191.0732421875, 820.8912353515625)
roughness_to

(512, 2048)
(11, 234.80126953125, 1521.9517822265625)
roughness_toa_refl_P080_*_B011.dat
/media/ehsan/6T_part1/2016/april_2016/14528_apr2016/project_april_2016_3cam/roughness_predicted_from_PH/roughness_toa_refl_P080_*_B011.dat
if more than 1 file found, there is a problem with this code and it has to be reviewed
1
(512, 2048)
(11, 134.5947265625, 1411.268798828125)
roughness_toa_refl_P080_*_B011.dat
/media/ehsan/6T_part1/2016/april_2016/14528_apr2016/project_april_2016_3cam/roughness_predicted_from_PH/roughness_toa_refl_P080_*_B011.dat
if more than 1 file found, there is a problem with this code and it has to be reviewed
1
(512, 2048)
(11, 37.3369140625, 1298.0015869140625)
roughness_toa_refl_P080_*_B011.dat
/media/ehsan/6T_part1/2016/april_2016/14528_apr2016/project_april_2016_3cam/roughness_predicted_from_PH/roughness_toa_refl_P080_*_B011.dat
if more than 1 file found, there is a problem with this code and it has to be reviewed
1
(512, 2048)
(10, 455.0888671875, 1182.228515625)
roug

...

if more than 1 file found, there is a problem with this code and it has to be reviewed
1
(512, 2048)
(27, 193.4384765625, 1537.8502197265625)
75
075
roughness_toa_refl_P075_*_B027.dat
/media/ehsan/6T_part1/2016/april_2016/14528_apr2016/project_april_2016_3cam/roughness_predicted_from_PH/roughness_toa_refl_P075_*_B027.dat
if more than 1 file found, there is a problem with this code and it has to be reviewed
1
(512, 2048)
(27, 88.2666015625, 1643.6107177734375)
76
076
roughness_toa_refl_P076_*_B027.dat
/media/ehsan/6T_part1/2016/april_2016/14528_apr2016/project_april_2016_3cam/roughness_predicted_from_PH/roughness_toa_refl_P076_*_B027.dat
if more than 1 file found, there is a problem with this code and it has to be reviewed
1
(512, 2048)
(26, 492.33203125, 1746.5791015625)
77
077
roughness_toa_refl_P077_*_B026.dat
/media/ehsan/6T_part1/2016/april_2016/14528_apr2016/project_april_2016_3cam/roughness_predicted_from_PH/roughness_toa_refl_P077_*_B026.dat
if more than 1 file found, there is a

(512, 2048)
(27, 395.0380859375, 1318.3837890625)
73
073
roughness_toa_refl_P073_*_B027.dat
/media/ehsan/6T_part1/2016/april_2016/14528_apr2016/project_april_2016_3cam/roughness_predicted_from_PH/roughness_toa_refl_P073_*_B027.dat
if more than 1 file found, there is a problem with this code and it has to be reviewed
1
(512, 2048)
(27, 295.5869140625, 1429.498046875)
74
074
roughness_toa_refl_P074_*_B027.dat
/media/ehsan/6T_part1/2016/april_2016/14528_apr2016/project_april_2016_3cam/roughness_predicted_from_PH/roughness_toa_refl_P074_*_B027.dat
if more than 1 file found, there is a problem with this code and it has to be reviewed
1
(512, 2048)
(27, 193.240234375, 1537.9708251953125)
75
075
roughness_toa_refl_P075_*_B027.dat
/media/ehsan/6T_part1/2016/april_2016/14528_apr2016/project_april_2016_3cam/roughness_predicted_from_PH/roughness_toa_refl_P075_*_B027.dat
if more than 1 file found, there is a problem with this code and it has to be reviewed
1
(512, 2048)
(27, 88.064453125, 1643.725

(512, 2048)
(28, 163.46875, 1033.7664794921875)
70
070
roughness_toa_refl_P070_*_B028.dat
/media/ehsan/6T_part1/2016/april_2016/14528_apr2016/project_april_2016_3cam/roughness_predicted_from_PH/roughness_toa_refl_P070_*_B028.dat
if more than 1 file found, there is a problem with this code and it has to be reviewed
1
(512, 2048)
(28, 73.080078125, 1152.3397216796875)
71
071
roughness_toa_refl_P071_*_B028.dat
/media/ehsan/6T_part1/2016/april_2016/14528_apr2016/project_april_2016_3cam/roughness_predicted_from_PH/roughness_toa_refl_P071_*_B028.dat
if more than 1 file found, there is a problem with this code and it has to be reviewed
1
(512, 2048)
(27, 491.611328125, 1204.5064697265625)
72
072
roughness_toa_refl_P072_*_B027.dat
/media/ehsan/6T_part1/2016/april_2016/14528_apr2016/project_april_2016_3cam/roughness_predicted_from_PH/roughness_toa_refl_P072_*_B027.dat
if more than 1 file found, there is a problem with this code and it has to be reviewed
1
(512, 2048)
(27, 395.123046875, 1318.18

if more than 1 file found, there is a problem with this code and it has to be reviewed
1
(512, 2048)
(28, 163.537109375, 1033.5640869140625)
70
070
roughness_toa_refl_P070_*_B028.dat
/media/ehsan/6T_part1/2016/april_2016/14528_apr2016/project_april_2016_3cam/roughness_predicted_from_PH/roughness_toa_refl_P070_*_B028.dat
if more than 1 file found, there is a problem with this code and it has to be reviewed
1
(512, 2048)
(28, 73.1533203125, 1152.1392822265625)
71
071
roughness_toa_refl_P071_*_B028.dat
/media/ehsan/6T_part1/2016/april_2016/14528_apr2016/project_april_2016_3cam/roughness_predicted_from_PH/roughness_toa_refl_P071_*_B028.dat
if more than 1 file found, there is a problem with this code and it has to be reviewed
1
(512, 2048)
(27, 491.6904296875, 1204.3079833984375)
72
072
roughness_toa_refl_P072_*_B027.dat
/media/ehsan/6T_part1/2016/april_2016/14528_apr2016/project_april_2016_3cam/roughness_predicted_from_PH/roughness_toa_refl_P072_*_B027.dat
if more than 1 file found, there 

if more than 1 file found, there is a problem with this code and it has to be reviewed
1
(512, 2048)
(28, 163.462890625, 1033.783447265625)
70
070
roughness_toa_refl_P070_*_B028.dat
/media/ehsan/6T_part1/2016/april_2016/14528_apr2016/project_april_2016_3cam/roughness_predicted_from_PH/roughness_toa_refl_P070_*_B028.dat
if more than 1 file found, there is a problem with this code and it has to be reviewed
1
(512, 2048)
(28, 73.0732421875, 1152.3565673828125)
71
071
roughness_toa_refl_P071_*_B028.dat
/media/ehsan/6T_part1/2016/april_2016/14528_apr2016/project_april_2016_3cam/roughness_predicted_from_PH/roughness_toa_refl_P071_*_B028.dat
if more than 1 file found, there is a problem with this code and it has to be reviewed
1
(512, 2048)
(27, 491.6044921875, 1204.5230712890625)
72
072
roughness_toa_refl_P072_*_B027.dat
/media/ehsan/6T_part1/2016/april_2016/14528_apr2016/project_april_2016_3cam/roughness_predicted_from_PH/roughness_toa_refl_P072_*_B027.dat
if more than 1 file found, there i

In [ ]:
# plot transect
transect_small["rms"].plot(x=transect_small.index)#, y=transect_small)
plt.plot(transect_from_misr)

In [ ]:
transect_from_misr

## statistics

In [ ]:
transect_small.describe()

## histogram

In [ ]:
transect_small["rms"].hist(bins=20, alpha=1)